In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [17]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(
    top_k_results=5,  # Get more results
    doc_content_chars_max=2000,  # More content
    load_max_docs=10,  # Load more documents
    continue_on_failure=True)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [18]:
tools=[wiki,arxiv]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/faizmemon/Documents/Langchain/venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=True, load_max_docs=10, load_all_available_meta=False, doc_content_chars_max=2000))]

In [20]:
import os

from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

#environment variables
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

E0000 00:00:1760444391.191713 5840239 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [21]:
# agent to use combined tools
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert research assistant specializing in comprehensive analysis and deep-dive research. You have access to multiple research tools:

🔬 AVAILABLE RESEARCH TOOLS:
1. Wikipedia Search: General knowledge and overview information
2. Enhanced Arxiv Search: Academic papers and research (5 results, detailed content)
3. Web Search: Current information and recent developments
4. Research Analysis Tool: Comprehensive multi-source analysis with configurable depth

📋 RESEARCH METHODOLOGY:
- Always start with the Research Analysis Tool for comprehensive overview
- Use specific tools for targeted information when needed
- Combine information from multiple sources for complete understanding
- Provide detailed citations and source references
- Analyze trends, patterns, and connections between sources
- Identify gaps in information and suggest further research directions

🎯 RESPONSE STRUCTURE:
1. Executive Summary (2-3 sentences)
2. Detailed Analysis (with sources)
3. Key Findings and Insights
4. Current Trends and Developments
5. Research Gaps and Future Directions
6. Source Citations

💡 ANALYSIS DEPTH LEVELS:
- Shallow: Quick overview (3 sources, 1000 chars)
- Medium: Standard analysis (5 sources, 2000 chars) 
- Deep: Comprehensive research (10 sources, 3000 chars)

Always aim for comprehensive, well-researched responses that provide deep insights into the topic."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create the agent
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools,max_iterations=5, verbose=True)

print("Agent created successfully!")
print("Available tools:", [tool.name for tool in tools])


Agent created successfully!
Available tools: ['wikipedia', 'arxiv']


In [23]:
# Jupyter-compatible interactive chat using widgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

class JupyterChat:
    def __init__(self, agent_executor):
        self.agent_executor = agent_executor
        self.conversation_history = []
        self.message_count = 0
        
        # Create widgets
        self.text_input = widgets.Text(
            placeholder='Type your question here...',
            description='Question:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='80%')
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary',
            icon='paper-plane'
        )
        
        self.clear_button = widgets.Button(
            description='Clear Chat',
            button_style='warning',
            icon='trash'
        )
        
        self.exit_button = widgets.Button(
            description='Exit Chat',
            button_style='danger',
            icon='sign-out'
        )
        
        self.output_area = widgets.Output()
        
        # Set up event handlers
        self.send_button.on_click(self.send_message)
        self.clear_button.on_click(self.clear_chat)
        self.exit_button.on_click(self.exit_chat)
        self.text_input.on_submit(self.send_message)
        
        # Display the chat interface
        self.display_chat()
    
    def display_chat(self):
        """Display the chat interface"""
        clear_output()
        
        # Welcome message
        print("🤖 Welcome to the Jupyter AI Agent Chat!")
        print("=" * 60)
        print("Available tools: Wikipedia,Arxiv Search")
        print("Type your questions below and click 'Send' or press Enter")
        print("=" * 60)
        
        # Display widgets
        display(widgets.HBox([self.text_input, self.send_button]))
        display(widgets.HBox([self.clear_button, self.exit_button]))
        display(self.output_area)
    
    def send_message(self, button=None):
        """Send a message to the agent"""
        user_input = self.text_input.value.strip()
        
        if not user_input:
            with self.output_area:
                print("🤖 Agent: Please enter a question.")
            return
        
        self.message_count += 1
        self.conversation_history.append(f"👤 You: {user_input}")
        
        with self.output_area:
            print(f"\n👤 You: {user_input}")
            print("🤖 Agent: Thinking...")
        
        try:
            # Get agent response
            result = self.agent_executor.invoke({"input": user_input})
            agent_response = result['output']
            
            self.conversation_history.append(f"🤖 Agent: {agent_response}")
            
            with self.output_area:
                print(f"🤖 Agent: {agent_response}")
                print("-" * 50)
        
        except Exception as e:
            error_msg = f"Sorry, I encountered an error: {str(e)}"
            self.conversation_history.append(f"🤖 Agent: {error_msg}")
            
            with self.output_area:
                print(f"🤖 Agent: {error_msg}")
                print("-" * 50)
        
        # Clear the input
        self.text_input.value = ''
    
    def clear_chat(self, button=None):
        """Clear the conversation history"""
        self.conversation_history = []
        self.message_count = 0
        
        with self.output_area:
            clear_output()
            print("🧹 Chat history cleared!")
    
    def exit_chat(self, button=None):
        """Exit the chat"""
        with self.output_area:
            print("\n🤖 Agent: Goodbye! Thanks for chatting!")
            print(f"📊 Total messages exchanged: {self.message_count}")
            print("=" * 60)
        
        # Disable widgets
        self.text_input.disabled = True
        self.send_button.disabled = True
        self.clear_button.disabled = True
        self.exit_button.disabled = True




In [11]:
# Create the chat instance
jupyter_chat = JupyterChat(agent_executor)

🤖 Welcome to the Jupyter AI Agent Chat!
Available tools: Wikipedia,Arxiv Search
Type your questions below and click 'Send' or press Enter


Output()